In [ ]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.1 MB/s eta 0:00:00


In [ ]:
!pip install diffusers

In [ ]:
!pip install ip_adapter

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline
from PIL import Image

from ip_adapter import IPAdapterXL

/usr/local/lib/python3.10/dist-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [ ]:
base_model_path = "stabilityai/stable-diffusion-xl-base-1.0"
image_encoder_path = "/content/IP-Adapter/sdxl_models/image_encoder"
ip_ckpt = "/content/IP-Adapter/sdxl_models/ip-adapter_sdxl.bin"
device = "cuda"

In [ ]:
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
# load SDXL pipeline
pipe = StableDiffusionXLPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    add_watermarker=False,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.78G [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [ ]:
import torch

# Empty the cache
torch.cuda.empty_cache()


In [ ]:
# load ip-adapter
ip_model = IPAdapterXL(pipe, image_encoder_path, ip_ckpt, device)

In [ ]:
# read image prompt
image = Image.open("/content/IP-Adapter/assets/PECORINO/000000.jpg")
image.resize((512, 512))

In [ ]:
# generate image variations with only image prompt
num_samples = 2
images = ip_model.generate(pil_image=image, num_samples=num_samples, num_inference_steps=30, seed=420)
grid = image_grid(images, 1, num_samples)
grid

NameError: name 'image' is not defined

In [ ]:
import random
import os

repertoire = "/content/IP-Adapter/prompts/"

list_of_cheese = []
with open(os.path.join(repertoire, "list_of_cheese.txt"), 'r') as f:
    list_of_cheese += f.readlines()

def choisir(fichier,repertoire,label):
    adjectifs = []
    lines = []
    with open(os.path.join(repertoire, fichier), 'r') as f:
        lines += f.readlines()
    for line in lines:
        line = line.split(",")
        line = [x.strip() for x in line]
        if label in line:
            for i in range(5):
                adjectifs.append(line[0])
        if "ALL" in line:
            adjectifs.append(line[0])
    return random.choice(adjectifs).strip()


#fichiers = [os.path.join(repertoire, 'adjectifs.txt'), os.path.join(repertoire, 'texture.txt'), os.path.join(repertoire, 'vue.txt'), os.path.join(repertoire, 'arriere_plan.txt')]
#syntaxe = "Créé une image de " + choisir_fromage('list_of_cheese.txt').lower()+" "+ choisir_adjectif('adjectifs.txt').lower()+ " et " + choisir_adjectif('adjectifs.txt').lower() + " qui est vu " +choisir_adjectif('vue.txt')+ ", placé sur " +choisir_adjectif('arriere_plan.txt')
#syntaxe_eng_simple = "Create an image of the cheese '" + choisir_fromage('list_of_cheese.txt').lower()+"'"
#syntaxe_eng_complex = "Create an image of the cheese '" + choisir_fromage('list_of_cheese.txt').lower()+"' "+ choisir_adjectif('adjectifs_eng.txt').lower()+ " and " + choisir_adjectif('adjectifs_eng.txt').lower() + " seen from " +choisir_adjectif('vue_eng.txt')+ ", placed on " +choisir_adjectif('arriere_plan_eng.txt')

def create_prompt(label, repertoire) :
    a = ("a picture of a" if random.random() > 0.5 else "a picture of a few")
    b = " " + choisir('before.txt', repertoire, label).lower()
    b2 = " " + choisir('before.txt', repertoire, label).lower()
    c = " " + label + " cheese"
    d = " " + choisir('after.txt', repertoire, label).lower()
    d2 = " " + choisir('after.txt', repertoire, label).lower()
    syntaxe = a
    p = 0.8
    p2 = 0.3
    if random.random() > 1-p:
        syntaxe += b
    if random.random() > 1-p2:
        syntaxe += b2
    if random.random() > 1-p:
        syntaxe += c
    if random.random() > 1-p2:
        syntaxe += d
    return syntaxe

In [ ]:
create_prompt('VACHERIN', "/content/IP-Adapter/prompts/")

'a picture of a traditional VACHERIN cheese'

In [ ]:
import os

# Chemin vers le dossier contenant les images
input_folder = "/content/IP-Adapter/assets/CHEVRE"
output_folder_drive = "/content/drive/My Drive/modal/CHEVRE/"

# Liste pour stocker les chemins des fichiers d'images
image_paths = []

# Parcourir le dossier pour récupérer les chemins des fichiers d'images
for file_name in os.listdir(input_folder):
    # Construire le chemin complet du fichier d'image
    image_path = os.path.join(input_folder, file_name)
        # Ajouter le chemin à la liste
    image_paths.append(image_path)

print(image_paths)

compteur = 0
# Générer de nouvelles images pour chaque image dans le dossier
for image_path in image_paths:
    # Charger l'image à partir du chemin du fichier
    image = Image.open(image_path)

    # Générer de nouvelles images à partir de l'image chargée
    num_samples = 2
    images = ip_model.generate(pil_image=image, num_samples=num_samples, num_inference_steps=30, seed=420, prompt, scale=0.3)


    # Enregistrer chaque nouvelle image dans le même dossier
    for x, img in enumerate(images):
        # Définir le chemin de fichier de sortie
        output_path = os.path.join(output_folder_drive, f"new_image_{compteur}_generated_2.png")

        # Enregistrer l'image générée
        img.save(output_path)
        compteur +=1


['/content/IP-Adapter/assets/CHEVRE/000004.jpg', '/content/IP-Adapter/assets/CHEVRE/000023.jpg', '/content/IP-Adapter/assets/CHEVRE/000010.jpg', '/content/IP-Adapter/assets/CHEVRE/000001.jpg', '/content/IP-Adapter/assets/CHEVRE/000000.jpg', '/content/IP-Adapter/assets/CHEVRE/000017.jpg', '/content/IP-Adapter/assets/CHEVRE/000024.jpg', '/content/IP-Adapter/assets/CHEVRE/000021.jpg', '/content/IP-Adapter/assets/CHEVRE/000008.jpg', '/content/IP-Adapter/assets/CHEVRE/000007.jpg', '/content/IP-Adapter/assets/CHEVRE/000013.jpg', '/content/IP-Adapter/assets/CHEVRE/000019.jpg', '/content/IP-Adapter/assets/CHEVRE/000015.jpg', '/content/IP-Adapter/assets/CHEVRE/000011.jpg', '/content/IP-Adapter/assets/CHEVRE/000002.jpg', '/content/IP-Adapter/assets/CHEVRE/000018.jpg', '/content/IP-Adapter/assets/CHEVRE/000016.jpg', '/content/IP-Adapter/assets/CHEVRE/000014.jpg', '/content/IP-Adapter/assets/CHEVRE/000012.jpg', '/content/IP-Adapter/assets/CHEVRE/000022.jpg', '/content/IP-Adapter/assets/CHEVRE/0000

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import random
import os


In [ ]:
output_folder_drive = "/content/drive/My Drive/modal/test7"

In [ ]:
# Traverse through all directories and subdirectories
dossier_principal = '/content/drive/MyDrive/modal/val'
repertoire = "/content/IP-Adapter/prompts/"

compteur = 0
for root, dirs, files in os.walk(dossier_principal):
    # Initialize a list to collect image paths and captions
    current_folder_name = os.path.basename(root).lower().replace(" ", "_")
    # Find all .jpg files in the current directory
    for file in files:
        image_path = os.path.join(root, file)
        image = Image.open(image_path)
        # Générer de nouvelles images à partir de l'image chargée
        num_samples = 3
        prompt = create_prompt(current_folder_name, repertoire)
        images = ip_model.generate(pil_image=image, num_samples=num_samples, num_inference_steps=30,prompt=prompt, seed=394)

        # Enregistrer chaque nouvelle image dans le même dossier
        for x, img in enumerate(images):
          # Définir le chemin de fichier de sortie,
          output_path = os.path.join(output_folder_drive, f"{current_folder_name}_{compteur}_generated_4.jpg")
          img.save(output_path)
          compteur += 1


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
output_folder_drive = "/content/drive/My Drive/modal/test6"
# Traverse through all directories and subdirectories
dossier_principal = '/content/drive/MyDrive/modal/val'
repertoire = "/content/IP-Adapter/prompts/"

compteur = 0
for root, dirs, files in os.walk(dossier_principal):
    # Initialize a list to collect image paths and captions
    current_folder_name = os.path.basename(root).lower().replace(" ", "_")
    # Find all .jpg files in the current directory
    for file in files:
        image_path = os.path.join(root, file)
        image = Image.open(image_path)
        # Générer de nouvelles images à partir de l'image chargée
        num_samples = 3
        #prompt = create_prompt(current_folder_name, repertoire)
        images = ip_model.generate(pil_image=image, num_samples=num_samples, num_inference_steps=30, seed=121)

        # Enregistrer chaque nouvelle image dans le même dossier
        for x, img in enumerate(images):
          # Définir le chemin de fichier de sortie
          output_path = os.path.join(output_folder_drive, f"{current_folder_name}_{compteur}_generated2.jpg")
          img.save(output_path)
          compteur += 1

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
output_folder_drive = "/content/drive/My Drive/modal/test7"
# Traverse through all directories and subdirectories
dossier_principal = '/content/drive/MyDrive/modal/val'
repertoire = "/content/IP-Adapter/prompts/"

compteur = 0
for root, dirs, files in os.walk(dossier_principal):
    # Initialize a list to collect image paths and captions
    current_folder_name = os.path.basename(root).lower().replace(" ", "_")
    # Find all .jpg files in the current directory
    for file in files:
        image_path = os.path.join(root, file)
        image = Image.open(image_path)
        # Générer de nouvelles images à partir de l'image chargée
        num_samples = 3
        #prompt = create_prompt(current_folder_name, repertoire)
        images = ip_model.generate(pil_image=image, num_samples=num_samples, num_inference_steps=30, seed=511)

        # Enregistrer chaque nouvelle image dans le même dossier
        for x, img in enumerate(images):
          # Définir le chemin de fichier de sortie
          output_path = os.path.join(output_folder_drive, f"{current_folder_name}_{compteur}_generated3.jpg")
          img.save(output_path)
          compteur += 1